In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
import networkx as nx
import numpy as np
from variation_utils import calculate_katz
from scipy.sparse.linalg import eigs
import scipy.sparse as sp

def get_edge_vector_from_adj_matrix(adj_matrix):
    sparse_matrix = sp.csr_matrix(adj_matrix)
    row_indices, col_indices = sparse_matrix.nonzero()
    edge_vector = np.vstack((row_indices, col_indices)).T
    return edge_vector

def generate_adjacency_matrix_and_katz_vector(n, p, seed=42, alpha=0.1, beta=1.0):
    G = nx.erdos_renyi_graph(n, p, directed=True, seed=seed)
    A_G = nx.adjacency_matrix(G).astype(np.float64)


    eigenvalues, _ = eigs(A_G, k=1, which='LM')  # 'LM': Largest Magnitude, tol is tolerance
    spectral_radius_G = np.abs(eigenvalues).max()
    print(f"Spectral Radius: {spectral_radius_G}, alpha_max = {1/spectral_radius_G}")

    katz_centrality = calculate_katz(A_G, alpha = alpha, beta=beta)

    # Reorder nodes based on Katz centrality
    sorted_indices = np.argsort(katz_centrality)
    sorted_nodes = np.array(G.nodes())[sorted_indices]

    # Generate the adjacency matrix
    adj_matrix = nx.adjacency_matrix(G, nodelist=sorted_nodes)

    sorted_katz_vector = katz_centrality[sorted_indices]

    assert np.allclose(sorted_katz_vector, calculate_katz(adj_matrix.astype(np.float64), alpha = alpha, beta=beta))
    return adj_matrix, sorted_katz_vector


In [2]:
n = 15
p = 4/n
adj_matrix, katz_vector = generate_adjacency_matrix_and_katz_vector(n=n, p=p, alpha=0.01, seed=49)
edges = get_edge_vector_from_adj_matrix(adj_matrix)

final_objective = greedy_search(katz_vector, edges, np.array([0]*len(katz_vector)), w=0.00009)
print(f"Final Objective: {final_objective}")

{0: [0, 2, 6, ...], 1: [1, 4, ...], 2: [3, 5, ...], 5: [7, 8, 9, ...]}


In [7]:
import numpy as np
from numba import njit


@njit
def compute_obj(v_K, A, clusters, w):
    """
    Compute the objective function value given the cluster assignment.

    Parameters:
    v_K (np.ndarray): The input data matrix (n_samples, n_features).
    A (np.ndarray): The matrix A in the objective function (n_features, n_features).
    clusters (list or np.ndarray): Cluster assignments for each data point (n_samples,).
    w (float): The weight parameter.

    Returns:
    float: The value of the objective function.
    """
    unique_clusters = np.unique(clusters)

    # Compute the norms for v_K and A
    norm_v_K = np.sum(v_K**2)

    sum_HvK = 0
    sum_HAT = 0

    for cluster in unique_clusters:
        indices = np.where(clusters == cluster)[0]
        n_j = len(indices)
        assert n_j > 0

        v_K_cluster_sum = np.sum(v_K[indices], axis=0)
        A_cluster_sum = np.sum(A.T[indices, :], axis=0)
        sum_HvK += (v_K_cluster_sum**2) / n_j
        sum_HAT += (np.sum(A_cluster_sum**2)) / n_j
    obj_value = norm_v_K - sum_HvK + w * sum_HAT
    return obj_value


@njit
def generate_unbalanced_splits(state):
    """
    Generate all possible unbalanced splits for each cluster.

    Parameters:
    state (np.ndarray): The current state of the clustering.

    Returns:
    list of np.ndarray: Each array represents a new state after an unbalanced split.
    """
    unique_clusters = np.unique(state)
    unbalanced_splits = []

    for cluster in unique_clusters:
        cluster_indices = np.where(state == cluster)[0]
        n = cluster_indices.size

        if n > 1:
            for m in range(1, n):
                indices_to_split = cluster_indices[:m]
                new_state = state.copy()
                new_cluster = np.max(state) + 1
                for idx in indices_to_split:
                    new_state[idx] = new_cluster
                unbalanced_splits.append(new_state)

    return unbalanced_splits


@njit
def generate_2_swaps(clusters, max_dist):
    """
    Generate all possible 2-swap moves.

    Parameters:
    state (np.ndarray): The current state of the clustering.
    max_dist (int): The maximum distance between two points that are swapped as defined by the order of points in `clusters`

    Returns:
    list of np.ndarray: Each array represents a new state after a 2-swap.
    """
    swaps = []
    n = clusters.size

    for i in range(n):
        for j in range(max(i - max_dist + 1, 0), min(i + max_dist, n)):
            if clusters[i] != clusters[j]:
                new_state = clusters.copy()
                new_state[i], new_state[j] = new_state[j], new_state[i]
                swaps.append(new_state)

    return swaps


@njit
def generate_1_swaps(clusters, max_dist):
    """
    Generate all possible 1-swap moves.
    I.e. Instead of doing a swap simply reassign the cluster of only one node with the other one that is within max_dist.

    Parameters:
    state (np.ndarray): The current state of the clustering.

    Returns:
    list of np.ndarray: Each array represents a new state after a 1-swap.
    """
    swaps = []
    n = clusters.size

    for i in range(n):
        for j in range(max(i - max_dist + 1, 0), min(i + max_dist, n)):
            if clusters[i] != clusters[j]:
                new_state = clusters.copy()
                new_state[i] = new_state[j]
                swaps.append(new_state)

    return swaps


@njit
def greedy_algorithm_inefficient(v_K, A, inital_clusters, w):
    """
    It is recommended v_K is sorted. (And A also in that same node order)
    """
    current_clusters = inital_clusters
    best_state = current_clusters
    best_clusters_objective = compute_obj(v_K, A, current_clusters, w)

    while True:
        best_next_clusters_objective = best_clusters_objective
        best_next_clusters = current_clusters

        # Go through all Unbalanced Split
        # for cluster in clusters: for possible unbalanced split in all possible splits: ... (update best_move and best_move_objective here)
        unbalanced_split_clusters = generate_unbalanced_splits(current_clusters)
        for one_swap_cluster in unbalanced_split_clusters:
            obj_value = compute_obj(v_K, A, one_swap_cluster, w)
            if obj_value < best_next_clusters_objective:
                best_next_clusters = one_swap_cluster
                best_next_clusters_objective = obj_value

        # Go through all 2-Swaps
        # one_swap_clusters = generate_2_swaps(current_clusters, two_swap_max_dist)
        # for one_swap_cluster in one_swap_clusters:
        #     obj_value = compute_obj(v_K, A, one_swap_cluster, w)
        #     if obj_value < best_next_clusters_objective:
        #         best_next_clusters = one_swap_cluster
        #         best_next_clusters_objective = obj_value

        # Go through all 1-Swaps
        # one_swap_clusters = generate_1_swaps(current_clusters, one_swap_max_dist)
        # for one_swap_cluster in one_swap_clusters:
        #     obj_value = compute_obj(v_K, A, one_swap_cluster, w)
        #     if obj_value < best_next_clusters_objective:
        #         best_next_clusters = one_swap_cluster
        #         best_next_clusters_objective = obj_value

        # best_next_state, best_next_state_objective
        if best_next_clusters_objective >= best_clusters_objective:
            break

        # Update the current state and the best objective found
        current_clusters = best_next_clusters
        best_state = current_clusters
        best_clusters_objective = best_next_clusters_objective

    return best_state, best_clusters_objective


In [8]:
n = 600
p = 4/n
adj_matrix, katz_vector = generate_adjacency_matrix_and_katz_vector(n=n, p=p, alpha=0.01)
edges = get_edge_vector_from_adj_matrix(adj_matrix)


Spectral Radius: 3.997412216688188, alpha_max = 0.25016184115945117
Katz converged after 10 iterations.
Katz converged after 10 iterations.


In [9]:
import time
import numpy as np

w = np.exp(-14)

# Measure the time taken by greedy_search
start_time = time.time()
best_state, best_clusters_objective = greedy_search(katz_vector, edges, np.array([0]*len(katz_vector)), w=w)
end_time = time.time()
greedy_search_time = end_time - start_time

print("Fast greedy_search done!")
# Measure the time taken by greedy_algorithm_inefficient
start_time = time.time()
best_state_slow, best_clusters_objective_slow = greedy_algorithm_inefficient(katz_vector, adj_matrix.todense(), np.array([0]*len(katz_vector)), w=w)
end_time = time.time()
greedy_algorithm_inefficient_time = end_time - start_time

#assert np.all(best_state == best_state_slow)
print("Best state:", best_state)
print("Time taken by greedy_search: {:.6f} seconds".format(greedy_search_time))
print("Time taken by greedy_algorithm_inefficient: {:.6f} seconds".format(greedy_algorithm_inefficient_time))


Iteration 0 Objective: -0.2861005177350486
Iteration 1 Objective: -0.3437942954009391
Iteration 2 Objective: -0.3910783243510716
Iteration 3 Objective: -0.401386163837071
Iteration 4 Objective: -0.4102531357077888
Iteration 5 Objective: -0.41830185116840174
Iteration 6 Objective: -0.4254485786900228
Iteration 7 Objective: -0.42772265124473396
Iteration 8 Objective: -0.42928322572487
Iteration 9 Objective: -0.43004394850829986
Iteration 10 Objective: -0.43019987514579333
Iteration 11 Objective: -0.4302197286524653
Iteration 12 Objective: -0.43023712393736746
Iteration 13 Objective: -0.4302538236943675
Iteration 14 Objective: -0.43026946363510876
Iteration 15 Objective: -0.4302757012355016
Iteration 16 Objective: -0.43028085356543005
Iteration 17 Objective: -0.43028571234476387
Iteration 18 Objective: -0.430287344448919
Iteration 19 Objective: -0.4302886646559873
Iteration 20 Objective: -0.43028992608733174
Iteration 21 Objective: -0.4302911333623
Iteration 22 Objective: -0.4302922394470

# Size stress test

In [10]:
n = 1000
p = 4/n
adj_matrix, katz_vector = generate_adjacency_matrix_and_katz_vector(n=n, p=p, alpha=0.01)
edges = get_edge_vector_from_adj_matrix(adj_matrix)

Spectral Radius: 3.997412216688187, alpha_max = 0.2501618411594512
Katz converged after 10 iterations.
Katz converged after 10 iterations.


In [12]:
%%prun -s cumtime
import time
import numpy as np

w = np.exp(-14)

# Measure the time taken by greedy_search
start_time = time.time()
best_state, best_clusters_objective = greedy_search(katz_vector, edges, np.array([0]*len(katz_vector)), w=w)
end_time = time.time()
greedy_search_time = end_time - start_time

print("Time taken by greedy_search: {:.6f} seconds".format(greedy_search_time))

Iteration 0 Objective: -0.2861005177350486
Iteration 1 Objective: -0.3437942954009391
Iteration 2 Objective: -0.3910783243510716
Iteration 3 Objective: -0.401386163837071
Iteration 4 Objective: -0.4102531357077888
Iteration 5 Objective: -0.41830185116840174
Iteration 6 Objective: -0.4254485786900228
Iteration 7 Objective: -0.42772265124473396
Iteration 8 Objective: -0.42928322572487
Iteration 9 Objective: -0.43004394850829986
Iteration 10 Objective: -0.43019987514579333
Iteration 11 Objective: -0.4302197286524653
Iteration 12 Objective: -0.43023712393736746
Iteration 13 Objective: -0.4302538236943675
Iteration 14 Objective: -0.43026946363510876
Iteration 15 Objective: -0.4302757012355016
Iteration 16 Objective: -0.43028085356543005
Iteration 17 Objective: -0.43028571234476387
Iteration 18 Objective: -0.430287344448919
Iteration 19 Objective: -0.4302886646559873
Iteration 20 Objective: -0.43028992608733174
Iteration 21 Objective: -0.4302911333623
Iteration 22 Objective: -0.4302922394470

         3222 function calls (3209 primitive calls) in 0.254 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.192    0.064 selectors.py:319(select)
        3    0.191    0.064    0.192    0.064 selectors.py:313(_select)
        4    0.000    0.000    0.041    0.010 events.py:82(_run)
        4    0.000    0.000    0.041    0.010 {method 'run' of '_contextvars.Context' objects}
        3    0.000    0.000    0.034    0.011 zmqstream.py:607(_handle_events)
        2    0.000    0.000    0.034    0.017 asyncio.py:200(_handle_events)
        3    0.000    0.000    0.033    0.011 zmqstream.py:648(_handle_recv)
        3    0.000    0.000    0.033    0.011 zmqstream.py:580(_run_callback)
        3    0.000    0.000    0.033    0.011 iostream.py:157(_handle_event)
        1    0.000    0.000    0.033    0.033 iostream.py:276(<lambda>)
        1    0.000    0.000    0.033    0.033 iostream.py:278(_

In [13]:
import os
os.environ['NUMBA_PROFILE'] = '1'


In [14]:
import numba
from numba import jit

@jit(nopython=True)
def my_function(x):
    y = 0
    for i in range(x):
        y += i
    return y


In [16]:
%%prun -s cumtime
result = my_function(1000000)
print(result)


499999500000
 

         146 function calls (143 primitive calls) in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        1    0.000    0.000    0.000    0.000 events.py:82(_run)
        1    0.000    0.000    0.000    0.000 {method 'run' of '_contextvars.Context' objects}
        1    0.000    0.000    0.000    0.000 asyncio.py:200(_handle_events)
        1    0.000    0.000    0.000    0.000 zmqstream.py:607(_handle_events)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 zmqstream.py:648(_handle_recv)
        2    0.000    0.000    0.000    0.000 attrsettr.py:42(__getattr__)
        1    0.000    0.000    0.000    0